In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url = "https://results.elections.virginia.gov/vaelections/2022%20June%20Republican%20Primary/Site/Member_House_of_Representatives_(02).html"
prefix = 'https://results.elections.virginia.gov/vaelections/2022%20June%20Republican%20Primary/Site' 

In [3]:
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
urls = []
for link in soup.find_all('a'):
    urls.append(link.get('href'))

In [4]:
county_urls = urls[-10:-2]

In [5]:
def create_vote_table(county_url):
    """
    """
    suffix = county_url
    curl = prefix + suffix[1:]
    # Extract county name
    county_string = county_url.lstrip('./Locality/').rstrip('/Member_House_of_Representatives_(02).html')
    # Parse the page
    creqs = requests.get(curl)
    csoup = BeautifulSoup(creqs.text, 'html.parser')
    # Find all headers
    headings = pd.DataFrame(csoup.find_all('span'))
    headings["id_string"] = headings[0].astype(str)
    jds = headings[headings["id_string"].str.contains('(02)')]
    # Craft a quick function to strip formatting and apply
    def remove_span(string): return string.lstrip("<span>").rstrip("</span>")
    jds["id_string"] = jds["id_string"].apply(remove_span)
    # load in tables from the page
    cpage = pd.read_html(curl)
    cdat = pd.concat(cpage,ignore_index=True)
    # Multiply the table by number of candidates
    jurisdiction = pd.DataFrame(np.repeat(jds.values, len(cdat.Candidate.unique()), axis=0))
    jurisdiction.columns = jds.columns
    # Merge the jurisdictions with vote shares
    vote = jurisdiction.join(cdat).drop(columns=[0])
    vote = vote.pivot(index='id_string', columns='Candidate', values='Votes').reset_index()
    #vote_share = vote.pivot(index='id_string', columns='Candidate', values='Percent').reset_index()
    #vote = vote_count.join(vote_share, lsuffix='_count', rsuffix='_share')
    #vote = vote.rename(columns={"id_string_count":"id_string"})
    vote["Precinct_#"] = vote["id_string"].str[:4]
    #vote = vote.drop(columns=["id_string_share"])
    vote["County"] = county_string
    return vote

In [6]:
# loop for all urls on the page
li = []
for county_url in county_urls:
    vote = create_vote_table(county_url)
    li.append(vote)
    
all_results = pd.concat(li)
all_results.to_csv("data/raw/va_cd2_2022.csv")

/Users/xavier/opt/anaconda3/lib/python3.8/site-packages/pandas/core/internals/construction.py:305: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  values = np.array([convert(v) for v in values])
/Users/xavier/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-5-8f917b2ac3a1>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jds["id_string"] = jds["id_string"].apply(remove_s

In [7]:
pd.read_csv("data/raw/va_prez_2016.csv")

,County/City,Ward,Pct,Donald J. Trump,Marco Rubio,Rafael Edward Cruz,John Kasich,"Benjamin S. Carson, Sr",Jeb Bush,Rand Paul,Mike Huckabee,Chris Christie,Carly Fiorina,Jim Gilmore,Lindsey Graham,Rick Santorum,All Others,Total Votes Cast
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Accomack County,-,101 - Chincoteague,328,107,66,30,63,1,1,1,1,0.0,1.0,0.0,0.0,0.0,599
2,Accomack County,-,201 - Atlantic,139,43,44,9,29,0,1,0,0,1.0,0.0,0.0,0.0,0.0,266
3,Accomack County,-,202 - Greenbackville,192,67,64,15,35,0,2,1,0,1.0,0.0,1.0,0.0,0.0,378
4,Accomack County,-,301 - Oak Hall,146,39,45,5,29,0,2,0,0,0.0,0.0,1.0,0.0,0.0,267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,York County,-,501 - Tabb,365,511,170,120,75,5,1,2,2,1.0,0.0,0.0,1.0,1.0,"1,254"
2743,York County,-,502 - Bethel,295,439,147,91,63,3,0,3,0,2.0,1.0,0.0,1.0,0.0,"1,045"
2744,York County,-,Ab - Central Absentee Precinct,82,96,48,41,17,11,5,0,3,0.0,0.0,0.0,0.0,0.0,303
2745,York County,-,Provisional,0,1,1,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2
